In [1]:
#importing important libraries and modules and defining dataframe
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
df = pd.read_csv('Iris.csv')
print(df)


      Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0      1            5.1           3.5            1.4           0.2   
1      2            4.9           3.0            1.4           0.2   
2      3            4.7           3.2            1.3           0.2   
3      4            4.6           3.1            1.5           0.2   
4      5            5.0           3.6            1.4           0.2   
..   ...            ...           ...            ...           ...   
145  146            6.7           3.0            5.2           2.3   
146  147            6.3           2.5            5.0           1.9   
147  148            6.5           3.0            5.2           2.0   
148  149            6.2           3.4            5.4           2.3   
149  150            5.9           3.0            5.1           1.8   

            Species  
0       Iris-setosa  
1       Iris-setosa  
2       Iris-setosa  
3       Iris-setosa  
4       Iris-setosa  
..              ...  
145  

In [2]:

#checking for null_values
null_values = df.isnull().sum()
#count null values for eachn column
print(null_values)
#as shown there are 24 null values in the Income Column

Id               0
SepalLengthCm    0
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     0
Species          0
dtype: int64


In [3]:
 

#Define training Set
X = df.iloc[:, 1:5].values  # All columns except the last one
y = df.iloc[:, -1].values   # Only the last column

#encoding the values of y to be 0,1,2 to represent each class as a number because models only deal with numbers
label_mapping = {"Iris-setosa": 0, "Iris-virginica": 1, "Iris-versicolor": 2}
y_enc = np.vectorize(label_mapping.get)(y)

# Displaying the shapes of X and y to verify and the values of y_enc
print("Shape of X:", X.shape)
print("Shape of y:", y_enc.shape)
print(y_enc)


Shape of X: (150, 4)
Shape of y: (150,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]


In [9]:
#NOTE: we do not scale this data because they are all in the same ranges, if not we'd be using standard scaler from
#scikit-learn to scale the data
#splitting training set for model evaluation after fitting is done to check for bias(underfitting) or variance(overfitting)
from sklearn.model_selection import train_test_split

# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables: x_ and y_.
x_train, x_, y_train, y_ = train_test_split(X, y_enc, test_size=0.40, random_state=1)

# Split the 40% subset above into two: one half for cross validation and the other for the test set
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_cv.shape}")
print(f"the shape of the cross validation set (target) is: {y_cv.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")


the shape of the training set (input) is: (90, 4)
the shape of the training set (target) is: (90,)

the shape of the cross validation set (input) is: (30, 4)
the shape of the cross validation set (target) is: (30,)

the shape of the test set (input) is: (30, 4)
the shape of the test set (target) is: (30,)


In [11]:
#increasing the training set size because the more training examples we have, the better our algorithm learns the training set
#so we duplicate the input and target array by a 1000 times
Xt = np.tile(x_train,(1000,1))
Yt= np.tile(y_train,(1000))   
print(Xt.shape, Yt.shape)   


(90000, 4) (90000,)
[0 1 1 ... 2 1 0]


In [12]:
#define tensorflow model with one hidden layer with three activations and activation function of relu(0 for the negative values
#of z and linear for the positive values of z )
#and an output layer with three activations representing the output's multiclass(i.e the three iris species) and an activation function of linear(values of
#z remain the sdme)
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        tf.keras.Input(shape=(4,)),
        Dense(3, activation='relu', name = 'layer1'),
        Dense(3, activation='linear', name = 'layer2')
     ]
)
model.summary()
#shows you a breakdown of the parameters

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 3)                 15        
_________________________________________________________________
layer2 (Dense)               (None, 3)                 12        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [13]:
#define the loss function. for better accuracy during compilation, we set the output activation to be linear instead of softmax
#and specified it in "from_logits = true"
#we define the learning rate and use the adam optimizer which dynamically adjusts the learning rate when training


model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)
#fit the algorithm to this data
model.fit(
    Xt,Yt,            
    epochs=10,
)

Train on 90000 samples
Epoch 1/10
90000/90000 [==============================] - 9s 96us/sample - loss: 0.2269
Epoch 2/10
90000/90000 [==============================] - 6s 70us/sample - loss: 0.0766
Epoch 3/10
90000/90000 [==============================] - 7s 79us/sample - loss: 0.0682
Epoch 4/10
90000/90000 [==============================] - 7s 79us/sample - loss: 0.0654
Epoch 5/10
90000/90000 [==============================] - 8s 84us/sample - loss: 0.0644
Epoch 6/10
90000/90000 [==============================] - 7s 82us/sample - loss: 0.0633
Epoch 7/10
90000/90000 [==============================] - 6s 71us/sample - loss: 0.0628
Epoch 8/10
90000/90000 [==============================] - 7s 79us/sample - loss: 0.0623
Epoch 9/10
90000/90000 [==============================] - 7s 72us/sample - loss: 0.0618
Epoch 10/10
90000/90000 [==============================] - 7s 83us/sample - loss: 0.0616


In [14]:
#getting the parameters(weight and bias) of the fit data
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[ 0.07043517 -0.25067753 -0.8253138 ]
 [-0.4743376  -0.12899333 -1.3761055 ]
 [ 0.01337022 -0.26141864  2.732112  ]
 [ 0.15794885  0.4977454   4.6687155 ]] 
b1: [ 0.         0.        -4.7897816]
W2:
 [[ -0.34927058  -0.7225988    0.2960987 ]
 [ -0.39196324  -0.32647347  -0.73963904]
 [-22.85709      2.7212315    0.03291858]] 
b2: [ 18.581818 -13.210989   6.718433]


In [25]:
#for model evaluation, using this paramters to predict the y labels for the training and cross validation set and checking for
#the percentage of errors. this will help us identify if the model has overfit or underfit i.e if the cross validation
#has a high rate of error (much greater than the training set) then this means the model has high variance and has overfit the 
#set
#however if the train set has very high rate of error then the model has high bias and has underfit

predictions_train = model.predict(x_train)
predictions_cv = model.predict(x_cv)
predictions_test = model.predict(x_test)
#make predictions(returns an array of numbers for each training example) on train, cross-validation and test set and categorize into classes
#based on highest number in array
def predict(predictions):
    m,n = predictions.shape
    yhat = np.zeros(m)
    for i in range(m):
        #iterating through each prediction (which is a list of numbers representing each class for each training example)
        #and predicting the class with the highest number as the class for that training example
        yhat[i] = np.argmax(predictions[i])
    return(yhat)
yhat_train = predict(predictions_train)
yhat_cv = predict(predictions_cv)
yhat_test = predict(predictions_test)


#checking for percentage of error for the train set and cv set
def check_error(yhat, y):
    count = 0
    for i in range(len(y)):
        if yhat[i] != y[i]:
            count +=1
    error = count/len(y)
    return error

err_train = check_error(yhat_train, y_train)
err_cv = check_error(yhat_cv, y_cv)
err_test = check_error(yhat_test, y_test)

print(err_train)
print(err_cv)
print(err_test)

0.03333333333333333
0.0
0.0


In [ ]:
#we can see the model does very well on the training set but does even better on the cv set having an accuracy of a 100%
#therefore we can confirm the model is a perfect fit 